### 필요 모듈 로드

In [1]:
import pandas as pd
import numpy as np

### 활성화 지표 파일 로드

https://city.go.kr/portal/notice/notice/view.do?nttId=7443

서울특별시 2020 활성화 지표 사용

필요한 열만 추출

In [2]:
active = pd.read_excel('도시쇠퇴현황(20.12월 기준)_공개용.xlsx',sheet_name='진단결과_총괄표',skiprows=16)
active = active[active['시도명'] == '서울특별시']
active['자치구'] = active.시군구명.str.split(' ').apply(lambda x: pd.Series(x)).rename(columns={1:'자치구'})['자치구']
active = active[['자치구', '읍면동명', '과거대비인구변화(%)', '최근인구변화(연속감소년수)', '과거대비사업체변화(%)',
       '최근사업체변화(연속감소년수)', '노후건축물비율(%)']]
active

,자치구,읍면동명,과거대비인구변화(%),최근인구변화(연속감소년수),과거대비사업체변화(%),최근사업체변화(연속감소년수),노후건축물비율(%)
0,종로구,사직동,-13.703589,2,0.338696,1,73.636364
1,종로구,삼청동,-71.395349,3,-2.691790,1,77.192982
2,종로구,부암동,-28.333333,5,4.251386,1,75.547703
3,종로구,평창동,-4.272303,4,2.092050,0,71.133568
4,종로구,무악동,-37.690513,3,-15.185185,4,54.545455
...,...,...,...,...,...,...,...
420,강동구,둔촌1동,-99.865471,5,-94.910180,5,70.370370
421,강동구,둔촌2동,-14.202147,3,-5.244253,3,63.047138
422,강동구,암사1동,-16.278442,3,-3.260870,2,62.685851
423,강동구,천호2동,-6.641188,3,-8.824392,1,74.177313


### 기초생활 수급자 파일 로드

https://data.seoul.go.kr/dataList/10113/S/2/datasetView.do

2020 서울시 국민기초생활보장 수급자 (동별) 통계 사용

txt 파일을 csv로 변환하여 저장 후 사용

In [3]:
basic_living = pd.read_csv('서울시 국민기초생활보장 수급자 (동별) 통계.csv',encoding='cp949')
basic_living = basic_living.iloc[3:,1:4]

기타, 소계 등 행정동과 관계없는 행 제거

In [4]:
basic_living = basic_living[basic_living['동']!='기타']
basic_living = basic_living[basic_living['동']!='소계']
basic_living

,자치구,동,총 수급자
4,종로구,사직동,109
5,종로구,삼청동,31
6,종로구,부암동,114
7,종로구,평창동,97
8,종로구,무악동,118
...,...,...,...
471,강동구,성내3동,483
472,강동구,둔촌2동,391
473,강동구,암사1동,"1,294"
474,강동구,천호2동,"1,439"


### 서울시 고령자현황 파일 로드

https://data.seoul.go.kr/dataList/10020/S/2/datasetView.do

서울시 고령자현황 (동별) 통계 사용

txt 파일을 csv로 변환하여 저장 후 사용

필요한 열만 추출

In [5]:
old = pd.read_csv('서울시 고령자현황 (동별) 통계.csv',encoding='cp949')
old = old.iloc[3:,[1,2,3,6]]

기타, 소계 등 행정동과 관계없는 행 제거

In [6]:
old = old[old['동']!='기타']
old = old[old['동']!='소계']
old

,자치구,동,전체인구,65세이상 인구
4,종로구,사직동,"9,806","1,843"
5,종로구,삼청동,"2,909",674
6,종로구,부암동,"10,333","1,839"
7,종로구,평창동,"18,582","3,398"
8,종로구,무악동,"8,592","1,460"
...,...,...,...,...
448,강동구,둔촌1동,262,97
449,강동구,둔촌2동,"26,413","4,135"
450,강동구,암사1동,"35,728","5,965"
451,강동구,천호2동,"35,251","5,969"


### 기초생활 수급자, 고령자현황 merge 후 비율 계산

In [7]:
people = pd.merge(old,basic_living,on=['자치구','동'],how='left')
people

,자치구,동,전체인구,65세이상 인구,총 수급자
0,종로구,사직동,"9,806","1,843",109
1,종로구,삼청동,"2,909",674,31
2,종로구,부암동,"10,333","1,839",114
3,종로구,평창동,"18,582","3,398",97
4,종로구,무악동,"8,592","1,460",118
...,...,...,...,...,...
420,강동구,둔촌1동,262,97,NaN
421,강동구,둔촌2동,"26,413","4,135",391
422,강동구,암사1동,"35,728","5,965","1,294"
423,강동구,천호2동,"35,251","5,969","1,439"


NaN 값이 존재하는 행(행정동)은 사용할 수 없을 것으로 판단

NaN 값 제거 - 둔촌1동

In [8]:
people[people['총 수급자'].isnull()]

,자치구,동,전체인구,65세이상 인구,총 수급자
420,강동구,둔촌1동,262,97,NaN


In [9]:
people = people.dropna().copy()

비율 계산을 위해 형변환 & 비율 계산

In [10]:
people['65세이상 인구'] = people['65세이상 인구'].str.replace(',','').astype('int64')
people['전체인구'] = people['전체인구'].str.replace(',','').astype('int64')
people['총 수급자'] = people['총 수급자'].str.replace(',','').astype('int64')

In [11]:
people['노인인구비율'] = people['65세이상 인구']/people['전체인구']
people['기초생활 대상자 비율'] = people['총 수급자']/people['전체인구']

In [12]:
people

,자치구,동,전체인구,65세이상 인구,총 수급자,노인인구비율,기초생활 대상자 비율
0,종로구,사직동,9806,1843,109,0.187946,0.011116
1,종로구,삼청동,2909,674,31,0.231695,0.010657
2,종로구,부암동,10333,1839,114,0.177973,0.011033
3,종로구,평창동,18582,3398,97,0.182865,0.005220
4,종로구,무악동,8592,1460,118,0.169926,0.013734
...,...,...,...,...,...,...,...
419,강동구,성내3동,23692,3701,483,0.156213,0.020387
421,강동구,둔촌2동,26413,4135,391,0.156552,0.014803
422,강동구,암사1동,35728,5965,1294,0.166956,0.036218
423,강동구,천호2동,35251,5969,1439,0.169329,0.040822


### 활성화 지표, 노인 인구 비율, 기초생활 대상자 비율 merge 최종 지표 생성

In [13]:
tmp = people.iloc[:,[0,1,5,6]]
tmp = tmp.rename(columns={'동':'읍면동명'})


활성화 지표와 읍면동명 통일 - 온점을 중간점으로 수정

ex) 종로1.2가동 -> 종로1·2가동

In [14]:
tmp['읍면동명'] = tmp['읍면동명'].apply(lambda x : x.replace('.','·'))

In [15]:
tmp

,자치구,읍면동명,노인인구비율,기초생활 대상자 비율
0,종로구,사직동,0.187946,0.011116
1,종로구,삼청동,0.231695,0.010657
2,종로구,부암동,0.177973,0.011033
3,종로구,평창동,0.182865,0.005220
4,종로구,무악동,0.169926,0.013734
...,...,...,...,...
419,강동구,성내3동,0.156213,0.020387
421,강동구,둔촌2동,0.156552,0.014803
422,강동구,암사1동,0.166956,0.036218
423,강동구,천호2동,0.169329,0.040822


In [16]:
active_plus = pd.merge(active,tmp,on=['자치구','읍면동명'],how='left')

기초생활 수급자 정보가 존재하지 않는 둔촌1동 제거

In [17]:
active_plus.isnull().sum()

자치구                0
읍면동명               0
과거대비인구변화(%)        0
최근인구변화(연속감소년수)     0
과거대비사업체변화(%)       0
최근사업체변화(연속감소년수)    0
노후건축물비율(%)         0
노인인구비율             1
기초생활 대상자 비율        1
dtype: int64

In [18]:
active_plus[active_plus['노인인구비율'].isnull()]

,자치구,읍면동명,과거대비인구변화(%),최근인구변화(연속감소년수),과거대비사업체변화(%),최근사업체변화(연속감소년수),노후건축물비율(%),노인인구비율,기초생활 대상자 비율
420,강동구,둔촌1동,-99.865471,5,-94.91018,5,70.37037,NaN,NaN


In [19]:
active_plus = active_plus.dropna().copy()

In [20]:
active_plus.isnull().sum()

자치구                0
읍면동명               0
과거대비인구변화(%)        0
최근인구변화(연속감소년수)     0
과거대비사업체변화(%)       0
최근사업체변화(연속감소년수)    0
노후건축물비율(%)         0
노인인구비율             0
기초생활 대상자 비율        0
dtype: int64

### 행정동 코드 merge

In [21]:
dong_cd = pd.read_csv('dong_cd.csv',encoding='cp949').rename(columns={'시군구명':'자치구'}).iloc[:,[0,2,3]]
dong_cd

,행정동코드,자치구,읍면동명
0,1111051500,종로구,청운효자동
1,1111053000,종로구,사직동
2,1111054000,종로구,삼청동
3,1111055000,종로구,부암동
4,1111056000,종로구,평창동
...,...,...,...
420,1174065000,강동구,성내2동
421,1174066000,강동구,성내3동
422,1174068500,강동구,길동
423,1174069000,강동구,둔촌1동


In [22]:
result = pd.merge(active_plus, dong_cd,on=['자치구','읍면동명'],how='left')
result = result[['자치구', '읍면동명', '행정동코드', '과거대비인구변화(%)', '최근인구변화(연속감소년수)', '과거대비사업체변화(%)',
       '최근사업체변화(연속감소년수)', '노후건축물비율(%)', '노인인구비율', '기초생활 대상자 비율']]

### 최종 활성화 지표

In [23]:
result

,자치구,읍면동명,행정동코드,과거대비인구변화(%),최근인구변화(연속감소년수),과거대비사업체변화(%),최근사업체변화(연속감소년수),노후건축물비율(%),노인인구비율,기초생활 대상자 비율
0,종로구,사직동,1111053000,-13.703589,2,0.338696,1,73.636364,0.187946,0.011116
1,종로구,삼청동,1111054000,-71.395349,3,-2.691790,1,77.192982,0.231695,0.010657
2,종로구,부암동,1111055000,-28.333333,5,4.251386,1,75.547703,0.177973,0.011033
3,종로구,평창동,1111056000,-4.272303,4,2.092050,0,71.133568,0.182865,0.005220
4,종로구,무악동,1111057000,-37.690513,3,-15.185185,4,54.545455,0.169926,0.013734
...,...,...,...,...,...,...,...,...,...,...
419,강동구,성내3동,1174066000,-14.897188,5,-0.754549,1,53.799392,0.156213,0.020387
420,강동구,둔촌2동,1174070000,-14.202147,3,-5.244253,3,63.047138,0.156552,0.014803
421,강동구,암사1동,1174057000,-16.278442,3,-3.260870,2,62.685851,0.166956,0.036218
422,강동구,천호2동,1174061000,-6.641188,3,-8.824392,1,74.177313,0.169329,0.040822


### 저장

In [24]:
result.to_csv('활성화지표_추가본.csv',encoding='cp949',index=False)